In [ ]:
import h5py
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from sklearn.metrics import roc_curve, auc
import tensorflow.keras.backend as K
import joblib

# compute a mask per event, and attach vec(0) for the first MLP output for those masked
# so S and pho steps are unchanged, but skipping first MLP computations
# the total dim N * D is still the same after the first MLP (those zero constituents are skipping phi)
# --> for zero-padded constituents, directly make D-dim zeros for them after the first MLP
# make a route:
# if mask == 0:
#     pass to first MLP
# else:
#     make D-dim zeros

with h5py.File('data.h5', 'r') as f:
    x = f['x'][:, :32, :]
    y = f['y'][:]
    
x[x[:, :, 0] < 2] = 0

q5 = np.percentile(x[:, :, 0], 5)
q95 = np.percentile(x[:, :, 0], 95)
x[:, :, 0] = (x[:, :, 0]) / (q95 - q5)

#mean = np.mean(x, axis=(0, 1), keepdims=True)  # Shape: (1, 1, 3)
#std = np.std(x, axis=(0, 1), keepdims=True)    # Shape: (1, 1, 3)
#x = (x - mean) / std

train_ratio = 0.6
val_ratio = 0.1
test_ratio = 1 - train_ratio - val_ratio
X_train_val, X_test, Y_train_val, Y_test = train_test_split(x, y, test_size = test_ratio, random_state = 42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size = val_ratio/(val_ratio + train_ratio), random_state = 42)
print('X_train shape: ' + str(X_train.shape))
print('X_val   shape: ' + str(X_val.shape))
print('X_test  shape: ' + str(X_test.shape))
print('Y_train shape: ' + str(Y_train.shape))
print('Y_val   shape: ' + str(Y_val.shape))
print('Y_test  shape: ' + str(Y_test.shape))
del X_train_val, Y_train_val

In [ ]:
def build_phi():
    input_constituent = layers.Input(shape=(3,))  # Each constituent has 3 features
    x = layers.Dense(32, activation='relu')(input_constituent)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    return models.Model(input_constituent, x, name="phi")

def build_rho():
    input_agg = layers.Input(shape=(32,))  # Aggregated max across constituents
    x = layers.Dense(32, activation='relu')(input_agg)
    output = layers.Dense(5, activation='softmax')(x)
    return models.Model(input_agg, output, name="rho")

phi = build_phi()

input_all_constituents = layers.Input(shape=(32, 3))

phi_outputs = layers.TimeDistributed(phi)(input_all_constituents)

agg = layers.GlobalMaxPooling1D()(phi_outputs)
#agg = layers.Flatten()(phi_outputs)

rho = build_rho()
final_output = rho(agg)

model = models.Model(input_all_constituents, final_output)

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0032),
              loss=tf.nn.softmax_cross_entropy_with_logits, metrics = 'accuracy')
model.summary()

In [ ]:
history = model.fit(X_train, Y_train,
                    validation_data = (X_val, Y_val),
                    epochs=10, batch_size=128)

In [ ]:
plt.figure(figsize = (15,10))
axes = plt.subplot(2, 2, 1)
axes.plot(history.history['loss'], label = 'train loss')
#axes.set_yscale(value = "log")
axes.plot(history.history['val_loss'], label = 'val loss')
axes.legend(loc = "upper right")
axes.set_xlabel('Epoch')
axes.set_ylabel('Loss')

In [ ]:
Y_pred = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1))))

In [ ]:
def plot_roc(y_test, y_pred, labels):
    for x, label in enumerate(labels):        
        fpr, tpr, _ = roc_curve(y_test[:, x], y_pred[:, x])
        plt.plot(fpr, tpr, label='{0} tagger, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle='-')
    #plt.semilogy()
    plt.ylabel("Signal Efficiency")
    plt.xlabel("Background Efficiency")
    #plt.ylim(0.001, 1)
    plt.grid(True)
    plt.legend(loc='best')  
    
plt.figure(figsize=(6, 6))
plot_roc(Y_test, Y_pred, ['g','q','w','z','t'])